# Installaciones de paquetes requeridos

- LIWC: se usa este paquete para poder cargar el diccionario provisto en el directorio ```data```

In [ ]:
#%pip install liwc

In [ ]:
#!python -m spacy download es_core_news_sm
#nltk.download('punkt')
#nltk.download('stopwords')

In [ ]:
import nltk
import string
import re
from collections import Counter
import pandas as pd
import spacy
from itertools import compress
import warnings
warnings.filterwarnings("ignore")
import liwc
parse, category_names = liwc.load_token_parser('data/dict_r.dic')

In [ ]:
nlp = spacy.load('es_core_news_sm')

In [ ]:
institutions = pd.read_excel('data/ELA - Instituciones.xlsx')

# Preprocesamiento

Este proceso se realiza con el fin de poder modificar o eliminar algunos componentes dentro de los corpus de texto:
- convierte todos los corpus a minuscula
- elimina signos de punctuacion
- elimina espacios en blanco dentro del corpus
- elimina posibles saltos de linea
- selecciona solo corpus que tienen mas de una palabra

In [ ]:
def preprocess(df, concept = None, all_less_concept = None):
  institutions = df
  if concept is not None:
    institutions = institutions[institutions["categoria"] == concept]

  if all_less_concept is not None:
    institutions = institutions[institutions["categoria"] != all_less_concept]

  institutions['fundamento'] = institutions['fundamento'].str.lower()
  institutions['fundamento'] = institutions['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
  institutions['fundamento'] = institutions['fundamento'].str.strip()
  institutions['fundamento'] = institutions['fundamento'].str.replace('\n', ' ')

  institutions = institutions[[len(str(x).split()) > 2 for x in institutions.fundamento]]
  return institutions.reset_index(drop=True)

# Eleccion de concepto target

Se elige el concepto de **Fuerzas Armadas** como concepto target para trabajar en el procesamiento, para ser cargado dentro del dataframe ```institutions_ffaa```

In [ ]:
institutions_ffaa = preprocess(institutions, "Fuerzas Armadas")

# Carga y preprocesamiento de todos los conceptos, menos el concepto target

Se cargan todos los conceptos, menos el concepto target (en este caso **Fuerzas Armadas**) para ser cargados en el dataframe ```institutions_less_ffaa```

In [ ]:
institutions_less_ffaa = preprocess(institutions, all_less_concept = "Fuerzas Armadas")

# Carga y preprocesamiento de todos los conceptos

Se cargan todos los conceptos, incluido el concepto target (en este caso **Fuerzas Armadas**) para ser cargados en el dataframe ```institutions_all```

In [ ]:
institutions_all = preprocess(institutions)

### 1 - Construccion de diccionario

Se construye el diccionario con el concepto target **Fuerzas Armadas** para parear las palabras de los corpus, con termino del diccionario cargado en ```data/dict_r.dic```.
Por cada corpus se cuenta la cantidad de veces que se repite el termino del diccionario.

In [ ]:
df = pd.DataFrame()
l_derechos = []

for ix in range(institutions_ffaa.shape[0]):
  texto = str(institutions_ffaa.fundamento[ix]).lower()
  doc = nlp(texto)
  tokens = [word.orth_ for word in doc]
  tcounts1 = Counter(category for token in tokens for category in parse(token))
  word_derechos = list(compress(tokens, ['derechos' in parse(token) for token in tokens]))
  df_counts = pd.DataFrame.from_dict([tcounts1]).reset_index()
  df_counts['index'] = ix
  df_counts['texto'] = texto
  df_counts['word_derechos'] =  ['' if len(word_derechos) == 0 else word_derechos]
  df = pd.concat([df,df_counts], axis=0, ignore_index=True)

  if len(word_derechos)>0:
        l_derechos.extend(word_derechos)

df_concept_ffaa = pd.DataFrame.from_dict(Counter(l_derechos), orient = 'index').reset_index()
df_concept_ffaa = df_concept_ffaa.rename(columns = {'index' : 'termino', 0 : 'count'})
df_concept_ffaa.sort_values('count', ascending = False)[0:20]

,termino,count
0,proteger,73
2,defender,58
9,respetar,25
8,cumplir,21
13,decir,15
7,ejercer,13
5,contribuir,12
3,responder,9
11,trabajar,5
12,procurar,5


### 2 (1) - Verificacion/Validacion diccionario

### Diccionario original

Se dejan los terminos cargados en el diccionario ```dict_r.dic``` y pareados con cada corpus del concepto **Fuerzas Armadas**. Con esto se obtiene la cantidad (ordenada de mayor a menor) de veces en que estos aparecen dentro de cada corpus procesado con el concepto target **Fuerzas Armadas**

In [ ]:
df_concept_ffaa.sort_values(by = ["count"], ascending = False)

,termino,count
0,proteger,73
2,defender,58
9,respetar,25
8,cumplir,21
13,decir,15
7,ejercer,13
5,contribuir,12
3,responder,9
11,trabajar,5
12,procurar,5


### 3 (1) - Desempeño diccionario

De los terminos propuestos en el procesamiento del punto **2 (1) - Verificacion/Validacion diccionario**, se genera una matriz de verificacion por todos los conceptos de la base de datos ```ELA - Instituciones.xlsx```. Todos los conceptos seran sumados al indicador, de acuerdo a cuantas veces se repita dicho termino del diccionario, dentro de la frase de cada concepto dentro de la base ```ELA - Instituciones.xlsx```

In [ ]:
df_dummy_categories_ffaa = pd.get_dummies(df_concept_ffaa.groupby(by = ['termino']).count().reset_index()['termino'])
df_dummy_categories_ffaa = df_dummy_categories_ffaa[0:0]
df_institutions_all_group = institutions_all.groupby(by = ['categoria']).count().reset_index()['categoria']
df_dummy_categories_ffaa['categoria'] = df_institutions_all_group

df_dummy_categories_ffaa = df_dummy_categories_ffaa.set_index("categoria")

for col in df_dummy_categories_ffaa.columns:
    df_dummy_categories_ffaa[col].values[:] = 0

for index, row in institutions_all.iterrows():
  for i, v in df_dummy_categories_ffaa.loc[row['categoria']].items():
    df_dummy_categories_ffaa.loc[row['categoria']][i] = df_dummy_categories_ffaa.loc[row['categoria']][i] + row['fundamento'].count(i)


KeyboardInterrupt



In [ ]:
df_dummy_categories_ffaa['suma'] = \
  df_dummy_categories_ffaa['apoyar'] + \
  df_dummy_categories_ffaa['conocer'] + \
  df_dummy_categories_ffaa['conservar'] + \
  df_dummy_categories_ffaa['contribuir'] + \
  df_dummy_categories_ffaa['cumplir'] + \
  df_dummy_categories_ffaa['decir'] + \
  df_dummy_categories_ffaa['defender'] + \
  df_dummy_categories_ffaa['educar'] + \
  df_dummy_categories_ffaa['ejercer'] + \
  df_dummy_categories_ffaa['guardar'] + \
  df_dummy_categories_ffaa['procurar'] + \
  df_dummy_categories_ffaa['proteger'] + \
  df_dummy_categories_ffaa['respetar'] + \
  df_dummy_categories_ffaa['responder'] + \
  df_dummy_categories_ffaa['servir'] + \
  df_dummy_categories_ffaa['tomar'] + \
  df_dummy_categories_ffaa['trabajar'] + \
  df_dummy_categories_ffaa['usar'] + \
  df_dummy_categories_ffaa['vivir']

df_dummy_categories_ffaa.sort_values(by = ['suma'], ascending = False)

_______________________________________________________________________________

### 2 (2) - Diccionario luego del procesamiento del punto 3 (Desempeño diccionario)
Luego del proceso del punto 3, se sacan los terminos:
- guardar
- responder
- tomar
- usar

In [ ]:
df_concept_ffaa_ = df_concept_ffaa.drop(df_concept_ffaa[df_concept_ffaa["termino"] == "vivir"].index)
df_concept_ffaa_ = df_concept_ffaa.drop(df_concept_ffaa[df_concept_ffaa["termino"] == "educar"].index)
df_concept_ffaa_ = df_concept_ffaa.drop(df_concept_ffaa[df_concept_ffaa["termino"] == "guardar"].index)
df_concept_ffaa_ = df_concept_ffaa.drop(df_concept_ffaa[df_concept_ffaa["termino"] == "conservar"].index)

df_concept_ffaa_.sort_values(by = ["count"], ascending = False)

### 3 (2) - Desempeño diccionario (con diccionario sin algunos terminos)

De los terminos propuestos en el procesamiento del punto 2 (2) - Verificacion/Validacion diccionario, se genera una matriz de verificacion por todos los conceptos de la base de datos ```ELA - Instituciones.xlsx```, con la savedad de eliminacion de terminos como **guardar**, **responder**, **tomar**, **usar**. Estos terminos son eliminados del procesamiento, tomando el significado bajo juicio experto, para poder tener terminos mas acordes a . Todos los conceptos seran sumados al indicador, de acuerdo a cuantas veces se repita dicho termino del diccionario, dentro de la frase de cada concepto dentro de la base ```ELA - Instituciones.xlsx```

In [ ]:
df_dummy_categories_ffaa_ = pd.get_dummies(df_concept_ffaa_.groupby(by = ['termino']).count().reset_index()['termino'])
df_dummy_categories_ffaa_ = df_dummy_categories_ffaa_[0:0]
df_institutions_all_group_ = institutions_all.groupby(by = ['categoria']).count().reset_index()['categoria']
df_dummy_categories_ffaa_['categoria'] = df_institutions_all_group_

df_dummy_categories_ffaa_ = df_dummy_categories_ffaa_.set_index("categoria")

for col in df_dummy_categories_ffaa_.columns:
    df_dummy_categories_ffaa_[col].values[:] = 0

for index, row in institutions_all.iterrows():
  for i, v in df_dummy_categories_ffaa_.loc[row['categoria']].items():
    df_dummy_categories_ffaa_.loc[row['categoria']][i] = df_dummy_categories_ffaa_.loc[row['categoria']][i] + row['fundamento'].count(i)



In [ ]:
df_dummy_categories_ffaa['suma'] = \
  df_dummy_categories_ffaa['apoyar'] + \
  df_dummy_categories_ffaa['conocer'] + \
  df_dummy_categories_ffaa['contribuir'] + \
  df_dummy_categories_ffaa['cumplir'] + \
  df_dummy_categories_ffaa['decir'] + \
  df_dummy_categories_ffaa['defender'] + \
  df_dummy_categories_ffaa['ejercer'] + \
  df_dummy_categories_ffaa['procurar'] + \
  df_dummy_categories_ffaa['proteger'] + \
  df_dummy_categories_ffaa['respetar'] + \
  df_dummy_categories_ffaa['responder'] + \
  df_dummy_categories_ffaa['servir'] + \
  df_dummy_categories_ffaa['tomar'] + \
  df_dummy_categories_ffaa['trabajar'] + \
  df_dummy_categories_ffaa['usar']

df_dummy_categories_ffaa.sort_values(by = ['suma'], ascending = False)